# Implement RAG chunking strategies with LangChain and watsonx.ai

**Author**: Anna Gutowska

In this tutorial, you will experiment with several chunking strategies using [LangChain](https://www.langchain.com/) and the latest IBM® [Granite™](https://www.ibm.com/granite) model now available on [watsonx.ai™](https://www.ibm.com/products/watsonx-ai). The overall goal will be to perform chunking to effective implement [retrieval augmented generation (RAG)](https://research.ibm.com/blog/retrieval-augmented-generation-RAG).

## What is chunking? 
[Chunking](https://www.ibm.com/architectures/papers/rag-cookbook/chunking) refers to the process of breaking large pieces of text into smaller text segments or chunks. To emphasize the importance of chunking, it is helpful to understand RAG. RAG is a technique in [natural language processing (NLP)](https://www.ibm.com/think/topics/natural-language-processing) that combines information retrieval and [large language models (LLMs)](https://www.ibm.com/think/topics/large-language-models) to retrieve relevant information from supplemental datasets to optimize the quality of the LLM’s output. To manage large documents, we can use chunking to split the text into smaller snippets of meaningful chunks. These text chunks can then be embedded and stored in a [vector database](https://www.ibm.com/think/topics/vector-database) through the use of an [embedding](https://www.ibm.com/think/topics/embedding) model. Finally, the RAG system can then uses semantic search to retrieve only the most relevant chunks. Smaller chunks tend to outperform larger chunks as they tend to be more manageable pieces for models of smaller [context window](https://www.ibm.com/think/topics/context-window) size. 

Some key components of chunking include: 
* Chunking strategy: Choosing the right chunking strategy for your RAG application is important as it determines the boundaries for setting chunks. We will explore some of these in the next section. 
* Chunk size: Maximum number of tokens to be in each chunk. Determining the appropriate chunk size usually involves some experimenting. 
* Chunk overlap: The number of tokens overlapping between chunks to preserve context. This is an optional parameter. 


## Choosing the right chunking strategy for your RAG application
There are several different chunking strategies to choose from. It is important to select the most effective chunking technique for the specific use case of your LLM application. Some commonly used chunking processes include:


* **Fixed-size chunking**: Splitting text based on a chunk size and optional chunk overlap. This approach is most common and straightforward.

* **Recursive chunking**: Iterating default separators until one of them produces the preferred chunk size. Default separators include `["\n\n", "\n", " ", ""]`. This chunking method uses hierarchical separators so that paragraphs, followed by sentences and then words, are kept together as much as possible.  

* **Semantic chunking**: Splitting text in a way that groups sentences based on the semantic similarity of their [embeddings](https://www.ibm.com/think/topics/embedding). Embeddings of high semantic similarity are closer together than those of low semantic similarity. This results in context-aware chunks.

* **Document-based chunking**: Splitting based on document structure. This splitter can utilize Markdown text, images, tables and even Python code classes and functions as ways of determining structure. In doing so, large documents can be chunked and processed by the LLM.

* **Agentic chunking**: Leverages [agentic AI](https://www.ibm.com/think/topics/ai-agents) by allowing the LLM to determine appropriate document splitting based on semantic meaning as well as content structure such as paragraph types, section headings, step-by-step instructions and more. This chunker is experimental and attempts to simulate human reasoning when processing long documents.

## Steps

### Step 1. Set up your environment
While you can choose from several tools, this tutorial walks you through how to set up an IBM account to use a Jupyter Notebook. 

1. Log in to [watsonx.ai](https://dataplatform.cloud.ibm.com/registration/stepone?context=wx&apps=all) using your IBM Cloud® account.

2. Create a [watsonx.ai project](https://www.ibm.com/docs/en/watsonx/saas?topic=projects-creating-project).

	You can get your project ID from within your project. Click the **Manage** tab. Then, copy the project ID from the **Details** section of the **General** page. You need this ID for this tutorial.

3. Create a [Jupyter Notebook](https://www.ibm.com/docs/en/watsonx/saas?topic=editor-creating-managing-notebooks).

	This step opens a Jupyter Notebook environment where you can copy the code from this tutorial.  Alternatively, you can download this notebook to your local system and upload it to your watsonx.ai project as an asset. To view more Granite tutorials, check out the [IBM Granite Community](https://github.com/ibm-granite-community). This tutorial is also available on [GitHub](https://github.com/IBM/ibmdotcom-tutorials/tree/main/generative-ai/rag-text-chunking-strategies.ipynb).


### Step 2. Set up a watsonx.ai Runtime instance and API key.

1. Create a [watsonx.ai Runtime](https://cloud.ibm.com/catalog/services/watsonxai-runtime) service instance (select your appropriate region and choose the Lite plan, which is a free instance).

2. Generate an application programming interface [(API) key](https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/ml-authentication.html). 

3. Associate the watsonx.ai Runtime service instance to the project that you created in [watsonx.ai](https://dataplatform.cloud.ibm.com/docs/content/wsj/getting-started/assoc-services.html). 

### Step 3. Install and import relevant libraries and set up your credentials

We need a few libraries and modules for this tutorial. Make sure to import the following ones and if they're not installed, a quick pip installation resolves the problem. 

*Note, this tutorial was built using Python 3.11.9.*

In [ ]:
# installations
!pip install -q langchain langchain-ibm langchain_experimental langchain-text-splitters langchain_chroma transformers bs4 langchain_huggingface sentence-transformers

In [ ]:
# imports 
import getpass

from langchain_ibm import WatsonxLLM
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from transformers import AutoTokenizer

To set our credentials, we need the `WATSONX_APIKEY` and `WATSONX_PROJECT_ID` you generated in step 1. We will also set the URL serving as the API endpoint.

In [ ]:
WATSONX_APIKEY = getpass.getpass("Please enter your watsonx.ai Runtime API key (hit enter): ")

WATSONX_PROJECT_ID = getpass.getpass("Please enter your project ID (hit enter): ")

URL = "https://us-south.ml.cloud.ibm.com"

### Step 4. Initialize your LLM

We will use Granite 3.1 as our LLM for this tutorial. To initialize the LLM, we need to set the model parameters. To learn more about these model parameters, such as the minimum and maximum token limits, refer to the [documentation](https://ibm.github.io/watsonx-ai-python-sdk/fm_model.html#metanames.GenTextParamsMetaNames).

In [ ]:
llm = WatsonxLLM(
    model_id= "ibm/granite-3-8b-instruct", 
    url=URL,
    apikey=WATSONX_APIKEY,
    project_id=WATSONX_PROJECT_ID,
    params={
        GenParams.DECODING_METHOD: "greedy",
        GenParams.TEMPERATURE: 0,
        GenParams.MIN_NEW_TOKENS: 5,
        GenParams.MAX_NEW_TOKENS: 2000,
        GenParams.REPETITION_PENALTY:1.2,
        GenParams.STOP_SEQUENCES: ["\n\n"]
    }
)

### Step 5. Load your document

The context we are using for our RAG pipeline is the official IBM announcement for the release of Granite 3.1. We can load the blog to a `Document` directly from the webpage by using LangChain's `WebBaseLoader`.

In [ ]:
url = "https://www.ibm.com/new/announcements/ibm-granite-3-1-powerful-performance-long-context-and-more"
doc = WebBaseLoader(url).load()

### Step 6. Perform text splitting

Let's provide sample code for implementing each of the chunking strategies we covered earlier in this tutorial available through LangChain. 

#### Fixed-size chunking
To implement fixed-size chunking, we can use LangChain's `CharacterTextSplitter` and set a `chunk_size` as well as `chunk_overlap`. The `chunk_size` is measured by the number of characters. Feel free to experiment with different values. We will also set the separator to be the newline character so that we can differentiate between paragraphs. For [tokenization](https://www.ibm.com/docs/en/watsonx/saas?topic=solutions-tokens), we can use the `granite-3.1-8b-instruct` tokenizer. The tokenizer breaks down text into tokens that can be processed by the LLM. 

In [ ]:
from langchain_text_splitters import CharacterTextSplitter

tokenizer = AutoTokenizer.from_pretrained("ibm-granite/granite-3.1-8b-instruct")
text_splitter = CharacterTextSplitter.from_huggingface_tokenizer(
    tokenizer, 
    separator="\n", #default: "\n\n"
    chunk_size=1200, 
    chunk_overlap=200)

fixed_size_chunks = text_splitter.create_documents([doc[0].page_content])

We can print one of the chunks for a better understanding of their structure.

In [ ]:
fixed_size_chunks[1]

Document(metadata={}, page_content='As always, IBM’s historical commitment to open source is reflected in the permissive and standard open source licensing for every offering discussed in this article.\n\r\n        Granite 3.1 8B Instruct: raising the bar for lightweight enterprise models\r\n    \nIBM’s efforts in the ongoing optimization the Granite series are most evident in the growth of its flagship 8B dense model. IBM Granite 3.1 8B Instruct now bests most open models in its weight class in average scores on the academic benchmarks evaluations included in the Hugging Face OpenLLM Leaderboard.\nThe evolution of the Granite model series has continued to prioritize excellence and efficiency in enterprise use cases, including agentic AI. This progress is most apparent in the newest 8B model’s significantly improved performance on IFEval, a dataset featuring tasks that test a model’s ability to follow detailed instructions, and Multi-step Soft Reasoning (MuSR), whose tasks measure reas

We can also use the tokenizer for verifying our process and to check the number of tokens present in each chunk. This step is optional and for demonstrative purposes. 

In [ ]:
for idx, val in enumerate(fixed_size_chunks):
    token_count = len(tokenizer.encode(val.page_content))
    print(f"The chunk at index {idx} contains {token_count} tokens.")

The chunk at index 0 contains 1106 tokens.
The chunk at index 1 contains 1102 tokens.
The chunk at index 2 contains 1183 tokens.
The chunk at index 3 contains 1010 tokens.


Great! It looks like our chunk sizes were appropriately implemented. 

#### Recursive chunking
For recursive chunking, we can use LangChain's `RecursiveCharacterTextSplitter`. Like the fixed-size chunking example, we can experiment with different chunk and overlap sizes.

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=0)
recursive_chunks = text_splitter.create_documents([doc[0].page_content])
recursive_chunks[:5]

[Document(metadata={}, page_content='IBM Granite 3.1: powerful performance, longer context and more'),
 Document(metadata={}, page_content='IBM Granite 3.1: powerful performance, longer context, new embedding models and more'),
 Document(metadata={}, page_content='Artificial Intelligence'),
 Document(metadata={}, page_content='Compute and servers'),
 Document(metadata={}, page_content='IT automation')]

The splitter successfully chunked the text by using the default separators: `["\n\n", "\n", " ", ""]`.

#### Semantic chunking
Semantic chunking requires an embedding or encoder model. We can use the `granite-embedding-30m-english` model as our embedding model. We can also print one of the chunks for a better understanding of their structure.

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_experimental.text_splitter import SemanticChunker

embeddings_model = HuggingFaceEmbeddings(model_name="ibm-granite/granite-embedding-30m-english")
text_splitter = SemanticChunker(embeddings_model)
semantic_chunks = text_splitter.create_documents([doc[0].page_content])
semantic_chunks[1]

Document(metadata={}, page_content='Our latest dense models (Granite 3.1 8B, Granite 3.1 2B), MoE models (Granite 3.1 3B-A800M, Granite 3.1 1B-A400M) and guardrail models (Granite Guardian 3.1 8B, Granite Guardian 3.1 2B) all feature a 128K token context length.We’re releasing a family of all-new embedding models. The new retrieval-optimized Granite Embedding models are offered in four sizes, ranging from 30M–278M parameters. Like their generative counterparts, they offer multilingual support across 12 different languages: English, German, Spanish, French, Japanese, Portuguese, Arabic, Czech, Italian, Korean, Dutch and Chinese. Granite Guardian 3.1 8B and 2B feature a new function calling hallucination detection capability, allowing increased control over and observability for agents making tool calls.All Granite 3.1, Granite Guardian 3.1, and Granite Embedding models are open source under Apache 2.0 license.These latest entries in the Granite series follow IBM’s recent launch of Docli

#### Document-based chunking
Documents of various file types are compatible with LangChain's document-based text splitters. For this tutorial's purposes, we will use a Markdown file. For examples of recursive JSON splitting, code splitting and HTML splitting, refer to the [LangChain documentation](https://python.langchain.com/docs/concepts/text_splitters/#document-structured-based).

An example of a Markdown file we can load is the README file for Granite 3.1 on IBM's [GitHub](https://github.com/DS4SD/docling/blob/main/README.md). 

In [ ]:
url = "https://raw.githubusercontent.com/ibm-granite/granite-3.1-language-models/refs/heads/main/README.md"
markdown_doc = WebBaseLoader(url).load()
markdown_doc

[Document(metadata={'source': 'https://raw.githubusercontent.com/ibm-granite/granite-3.1-language-models/refs/heads/main/README.md'}, page_content='\n\n\n\n  :books: Paper (comming soon)\xa0 | :hugs: HuggingFace Collection\xa0 | \n  :speech_balloon: Discussions Page\xa0 | ðŸ“˜ IBM Granite Docs\n\n\n---\n## Introduction to Granite 3.1 Language Models\nGranite 3.1 language models are lightweight, state-of-the-art, open foundation models that natively support multilinguality, coding, reasoning, and tool usage, including the potential to be run on constrained compute resources. All the models are publicly released under an Apache 2.0 license for both research and commercial use. The models\' data curation and training procedure were designed for enterprise usage and customization, with a process that evaluates datasets for governance, risk and compliance (GRC) criteria, in addition to IBM\'s standard data clearance process and document quality checks.\n\nGranite 3.1 language models extend 

Now, we can use LangChain's `MarkdownHeaderTextSplitter` to split the file by header type, which we set in the `headers_to_split_on` list. We will also print one of the chunks as an example.

In [ ]:
#document based chunking
from langchain_text_splitters import MarkdownHeaderTextSplitter

headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
]

markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on)
document_based_chunks = markdown_splitter.split_text(markdown_doc[0].page_content)
document_based_chunks[3]

Document(metadata={'Header 2': 'How to Use our Models?', 'Header 3': 'Inference'}, page_content='This is a simple example of how to use Granite-3.1-1B-A400M-Instruct model.  \n```python\nimport torch\nfrom transformers import AutoModelForCausalLM, AutoTokenizer\n\ndevice = "auto"\nmodel_path = "ibm-granite/granite-3.1-1b-a400m-instruct"\ntokenizer = AutoTokenizer.from_pretrained(model_path)\n# drop device_map if running on CPU\nmodel = AutoModelForCausalLM.from_pretrained(model_path, device_map=device)\nmodel.eval()\n# change input text as desired\nchat = [\n{ "role": "user", "content": "Please list one IBM Research laboratory located in the United States. You should only output its name and location." },\n]\nchat = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)\n# tokenize the text\ninput_tokens = tokenizer(chat, return_tensors="pt").to(device)\n# generate output tokens\noutput = model.generate(**input_tokens,\nmax_new_tokens=100)\n# decode output toke

As you can see in the output, the chunking successfully split the text by header type. 

### Step 7. Create vector store

Now that we have experimented with various chunking strategies, let's move along with our RAG implementation. For this tutorial, we will choose the chunks produced by the semantic split and convert them to vector embeddings. An open source vector store we can use is [Chroma DB](https://python.langchain.com/docs/integrations/vectorstores/chroma/). We can easily access Chroma functionality through the `langchain_chroma` package.

Let's initialize our Chroma vector database, provide it with our embeddings model and add our documents produced by semantic chunking.

In [ ]:
vector_db = Chroma(
    collection_name="example_collection",
    embedding_function=embeddings_model,
    persist_directory="./chroma_langchain_db",  # Where to save data locally, remove if not necessary
)

In [ ]:
vector_db.add_documents(semantic_chunks)

['84fcc1f6-45bb-4031-b12e-031139450cf8',
 '433da718-0fce-4ae8-a04a-e62f9aa0590d',
 '4bd97cd3-526a-4f70-abe3-b95b8b47661e',
 '342c7609-b1df-45f3-ae25-9d9833829105',
 '46a452f6-2f02-4120-a408-9382c240a26e']

### Step 8. Structure the prompt template
Next, we can move onto creating a prompt template for our LLM. This prompt template allows us to ask multiple questions without altering the initial prompt structure. We can also provide our vector store as the retriever. This step finalizes the RAG structure. 

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.prompts import PromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt_template = """<|start_of_role|>user<|end_of_role|>Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {input}<|end_of_text|>
<|start_of_role|>assistant<|end_of_role|>"""

qa_chain_prompt = PromptTemplate.from_template(prompt_template)
combine_docs_chain = create_stuff_documents_chain(llm, qa_chain_prompt)
rag_chain = create_retrieval_chain(vector_db.as_retriever(), combine_docs_chain)

### Step 9. Prompt the RAG chain

Using our completed RAG workflow, let's invoke a user query. First, we can strategically prompt the model without any additional context from the vector store we built to test whether the model is using its built-in knowledge or truly using the RAG context. The Granite 3.1 announcement blog references [Docling](https://github.com/DS4SD/docling?tab=readme-ov-file), IBM's tool for parsing various document types and converting them into Markdown or JSON. Let's ask the LLM about Docling.

In [ ]:
output = llm.invoke("What is Docling?")
output

"\nDocling is a platform that allows users to create, share and discover interactive documents. It's like having your own personal library of dynamic content where you can add notes, highlights, bookmarks, and even collaborate with others in real-time. Think of it as the next generation of document management systems designed for modern collaboration needs."

Clearly, the model was not trained on information about Docling and without outside tools or information, it cannot provide us with the correct information. The model hallucinates. Now, let's try providing the same query to the RAG chain we built. 

In [ ]:
rag_output = rag_chain.invoke({"input": "What is Docling?"})
rag_output['answer']

'Docling is a powerful tool developed by IBM Deep Search for parsing documents in various formats such as PDF, DOCX, images, PPTX, XLSX, HTML, and AsciiDoc, and converting them into model-friendly formats like Markdown or JSON. This enables easier access to the information within these documents for models like Granite for tasks such as RAG and other workflows. Docling is designed to integrate seamlessly with agentic frameworks like LlamaIndex, LangChain, and Bee, providing developers with the flexibility to incorporate its assistance into their preferred ecosystem. It surpasses basic optical character recognition (OCR) and text extraction methods by employing advanced contextual and element-based preprocessing techniques. Currently, Docling is open-sourced under the permissive MIT License, and the team continues to develop additional features, including equation and code extraction, as well as metadata extraction.'

Great! The Granite model correctly used the RAG context to tell us correct information about Docling while preserving semantic coherence. We proved this same result was not possible without the use of RAG. 

## Summary
In this tutorial, you created a RAG pipeline and experimented with several chunking strategies to improve the system’s retrieval accuracy. Using the Granite 3.1 model, we successfully produced appropriate model responses to a user query related to the documents provided as context. The text we used for this RAG implementation was loaded from a blog on ibm.com announcing the release of Granite 3.1. The model provided us with information only accessible through the provided context since it was not part of the model's initial knowledge base. 

For those in search of further reading, check out the results of a [project](https://developer.ibm.com/articles/awb-enhancing-llm-performance-document-chunking-with-watsonx/) comparing LLM performance using HTML structured chunking in comparison to watsonx chunking.